In [4]:
import numpy as np
import os
import glob
np.random.seed(1111)

In [5]:
#files = glob.glob("./ExtractedFeatures/csv/*")
files = glob.glob("./ExtractedFeatures/csv/2/*")
print("Total number of files: ", len(files))
print("Showing first 10 files...")
files[:10]

Total number of files:  2700
Showing first 10 files...


['./ExtractedFeatures/csv/2/2_450.csv',
 './ExtractedFeatures/csv/2/2_1864.csv',
 './ExtractedFeatures/csv/2/2_336.csv',
 './ExtractedFeatures/csv/2/2_322.csv',
 './ExtractedFeatures/csv/2/2_1870.csv',
 './ExtractedFeatures/csv/2/2_444.csv',
 './ExtractedFeatures/csv/2/2_2389.csv',
 './ExtractedFeatures/csv/2/2_1680.csv',
 './ExtractedFeatures/csv/2/2_1858.csv',
 './ExtractedFeatures/csv/2/2_1694.csv']

In [6]:
import pandas as pd
import re  

In [10]:
import tensorflow as tf
print(tf.__version__)

2.0.0-alpha0


In [15]:

def tf_data_generator(file_list, batch_size = 20):
    i = 0
    while True:
        if i*batch_size >= len(file_list):  
            i = 0
            np.random.shuffle(file_list)
        else:
            file_chunk = file_list[i*batch_size:(i+1)*batch_size] 
            data = []
            labels = []
            label_classes = tf.constant(["0", "1", "2"])
            for file in file_chunk:
                #temp = np.genfromtxt(file, max_rows=4000, dtype=float)
                temp = pd.read_csv(open(file,'r'), sep='\s+',skiprows=0,nrows=4500,usecols = [0,1,2,3,4]) # Change this lfault_foldersne to read any other type of file
                temp = temp.dropna()
                #print(temp.shape)
                #print(temp[:10])
                temp = temp[:4000]
                #temp = np.squeeze(np.asarray(temp))
                data.append(temp.values.reshape(4000,5,1)) # Convert column data to matrix like data with one channel
                pattern = tf.constant(eval("file[24:25]"))      # Pattern extracted from file_name
                for j in range(len(label_classes)):
                    if re.match(pattern.numpy(), label_classes[j].numpy()): 
                        labels.append(j)  
            data = np.asarray(data).reshape(-1,4000,5,1)
            labels = np.asarray(labels)
            yield data, labels
            i = i + 1

In [16]:
check_data = tf_data_generator(files, batch_size = 10)

In [17]:
num = 0
for data, labels in check_data:
    print(data.shape, labels.shape)
    print(labels, "<--Labels")
    print()
    num = num + 1
    if num > 5: break

(10, 4000, 5, 1) (10,)
[2 2 2 2 2 2 2 2 2 2] <--Labels

(10, 4000, 5, 1) (10,)
[2 2 2 2 2 2 2 2 2 2] <--Labels

(10, 4000, 5, 1) (10,)
[2 2 2 2 2 2 2 2 2 2] <--Labels

(10, 4000, 5, 1) (10,)
[2 2 2 2 2 2 2 2 2 2] <--Labels

(10, 4000, 5, 1) (10,)
[2 2 2 2 2 2 2 2 2 2] <--Labels

(10, 4000, 5, 1) (10,)
[2 2 2 2 2 2 2 2 2 2] <--Labels



In [18]:
print(data)


[[[['-0.16']
   ['0.38']
   ['0.07']
   [0.24]
   [-0.18]]

  [['-0.16']
   ['0.38']
   ['0.07']
   [0.24]
   [-0.18]]

  [['-0.15']
   ['0.38']
   ['0.07']
   [0.24]
   [-0.18]]

  ...

  [['0.94']
   ['-0.09']
   ['0.23']
   [0.15]
   [0.12]]

  [['0.93']
   ['-0.09']
   ['0.23']
   [0.15]
   [0.12]]

  [['0.92']
   ['-0.09']
   ['0.23']
   [0.15]
   [0.12]]]


 [[['1.19']
   ['0.99']
   ['0.80']
   [2.18]
   [4.25]]

  [['1.20']
   ['0.99']
   ['0.80']
   [2.18]
   [4.25]]

  [['1.20']
   ['0.99']
   ['0.80']
   [2.18]
   [4.25]]

  ...

  [['0.29']
   ['0.04']
   ['-0.85']
   [-0.3]
   [0.43]]

  [['0.29']
   ['0.03']
   ['-0.85']
   [-0.29]
   [0.43]]

  [['0.29']
   ['0.03']
   ['-0.85']
   [-0.29]
   [0.44]]]


 [[['3046793.92']
   ['131889.93']
   ['56944.03']
   [18120.35]
   [16102.58]]

  [['3042372.06']
   ['131961.08']
   ['56910.23']
   [18124.14]
   [16107.67]]

  [['3033528.38']
   ['131977.08']
   ['56890.00']
   [18132.06]
   [16104.88]]

  ...

  [['579966.02']
   ['

In [19]:
batch_size = 15
dataset = tf.data.Dataset.from_generator(tf_data_generator,args= [files, batch_size],output_types = (tf.float32, tf.float32),
                                                output_shapes = ((None,4000,5,1),(None,)))

W0724 20:58:21.721873 4560561600 deprecation.py:323] From /anaconda3/envs/tensorflow/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:410: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


In [20]:
print(dataset)

<DatasetV1Adapter shapes: ((None, 4000, 5, 1), (None,)), types: (tf.float32, tf.float32)>


In [21]:
num = 0
for data, labels in dataset:
    print(data.shape, labels.shape)
    print(labels)
    print()
    num = num + 1
    if num > 7: break

(15, 4000, 5, 1) (15,)
tf.Tensor([2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.], shape=(15,), dtype=float32)

(15, 4000, 5, 1) (15,)
tf.Tensor([2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.], shape=(15,), dtype=float32)

(15, 4000, 5, 1) (15,)
tf.Tensor([2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.], shape=(15,), dtype=float32)

(15, 4000, 5, 1) (15,)
tf.Tensor([2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.], shape=(15,), dtype=float32)

(15, 4000, 5, 1) (15,)
tf.Tensor([2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.], shape=(15,), dtype=float32)

(15, 4000, 5, 1) (15,)
tf.Tensor([2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.], shape=(15,), dtype=float32)

(15, 4000, 5, 1) (15,)
tf.Tensor([2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.], shape=(15,), dtype=float32)

(15, 4000, 5, 1) (15,)
tf.Tensor([2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2.], shape=(15,), dtype=float32)



In [121]:
import shutil

In [122]:
fault_folders = ["0", "1", "2"]
for folder_name in fault_folders:
    os.mkdir(os.path.join("./ExtractedFeatures/csv", folder_name))

In [124]:
for file in files:
    pattern = "^" + eval("file[24:25]")
    for j in range(len(fault_folders)):
        if re.match(pattern, fault_folders[j]):
            dest = os.path.join("./ExtractedFeatures/csv",eval("fault_folders[j]"))
            shutil.move(file, dest)

Error: Destination path './ExtractedFeatures/csv/0/0_780.csv' already exists

In [22]:
glob.glob("./ExtractedFeatures/csv/*")

['./ExtractedFeatures/csv/0',
 './ExtractedFeatures/csv/1',
 './ExtractedFeatures/csv/2']

In [23]:
label_0_files = glob.glob("./ExtractedFeatures/csv/0/*")
label_1_files = glob.glob("./ExtractedFeatures/csv/1/*")
label_2_files = glob.glob("./ExtractedFeatures/csv/2/*")


In [24]:
from sklearn.model_selection import train_test_split

In [25]:
label_0_train, label_0_test = train_test_split(label_0_files, test_size = .2, random_state = 5)
label_1_train, label_1_test = train_test_split(label_1_files, test_size = .2, random_state = 55)
label_2_train, label_2_test = train_test_split(label_2_files, test_size = .2, random_state = 555)


In [26]:
label_0_train, label_0_val = train_test_split(label_0_train, test_size = .1, random_state = 5)
label_1_train, label_1_val = train_test_split(label_1_train, test_size = .1, random_state = 55)
label_2_train, label_2_val = train_test_split(label_2_train, test_size = .1, random_state = 555)


In [27]:
train_file_names = label_0_train + label_1_train + label_2_train 
validation_file_names = label_0_val + label_1_val + label_2_val 
test_file_names = label_0_test + label_1_test + label_2_test 

# Shuffle data (We don't need to shuffle validation and test data)
np.random.shuffle(train_file_names)

In [28]:
print("Number of train_files:" ,len(train_file_names))
print("Number of validation_files:" ,len(validation_file_names))
print("Number of test_files:" ,len(test_file_names))

Number of train_files: 5832
Number of validation_files: 648
Number of test_files: 1620


In [29]:
batch_size = 10
train_dataset = tf.data.Dataset.from_generator(tf_data_generator, args = [train_file_names, batch_size], 
                                              output_shapes = ((None,4000,5,1),(None,)),
                                              output_types = (tf.float32, tf.float32))

validation_dataset = tf.data.Dataset.from_generator(tf_data_generator, args = [validation_file_names, batch_size],
                                                   output_shapes = ((None,4000,5,1),(None,)),
                                                   output_types = (tf.float32, tf.float32))

test_dataset = tf.data.Dataset.from_generator(tf_data_generator, args = [test_file_names, batch_size],
                                             output_shapes = ((None,4000,5,1),(None,)),
                                             output_types = (tf.float32, tf.float32))


In [30]:
from tensorflow.keras import layers

In [31]:

model = tf.keras.Sequential([
    layers.Conv2D(16, (3,3), activation = "relu", input_shape = (4000,5,1)),
    layers.MaxPool2D(2,2),
    #layers.Conv2D(32, 5, activation = "relu"),
    #layers.MaxPool2D(2),
    layers.Flatten(),
    layers.Dense(16, activation = "relu"),
    layers.Dense(3, activation = "softmax")
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 3998, 3, 16)       160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 1999, 1, 16)       0         
_________________________________________________________________
flatten (Flatten)            (None, 31984)             0         
_________________________________________________________________
dense (Dense)                (None, 16)                511760    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 51        
Total params: 511,971
Trainable params: 511,971
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

In [33]:
steps_per_epoch = np.int(np.ceil(len(train_file_names)/batch_size))
validation_steps = np.int(np.ceil(len(validation_file_names)/batch_size))
steps = np.int(np.ceil(len(test_file_names)/batch_size))
print("steps_per_epoch = ", steps_per_epoch)
print("validation_steps = ", validation_steps)
print("steps = ", steps)

steps_per_epoch =  584
validation_steps =  65
steps =  162


In [34]:
model.fit(train_dataset, validation_data = validation_dataset, steps_per_epoch = steps_per_epoch,
         validation_steps = validation_steps, epochs = 10)

Epoch 1/10
584/584 [==============================] - 124s 212ms/step - loss: 294653.5270 - accuracy: 0.3326 - val_loss: 352512.4340 - val_accuracy: 0.3333
Epoch 2/10
584/584 [==============================] - 117s 200ms/step - loss: 39315.7713 - accuracy: 0.3230 - val_loss: 310972.3740 - val_accuracy: 0.3333
Epoch 3/10
584/584 [==============================] - 118s 203ms/step - loss: 1.0966 - accuracy: 0.3352 - val_loss: 310970.5238 - val_accuracy: 0.3333
Epoch 4/10
584/584 [==============================] - 116s 198ms/step - loss: 6379.4747 - accuracy: 0.3318 - val_loss: 259011.6973 - val_accuracy: 0.3318
Epoch 5/10
584/584 [==============================] - 119s 204ms/step - loss: 1.0957 - accuracy: 0.3261 - val_loss: 259011.7600 - val_accuracy: 0.3426
Epoch 6/10
584/584 [==============================] - 115s 197ms/step - loss: 1.0965 - accuracy: 0.3354 - val_loss: 259011.2920 - val_accuracy: 0.3380
Epoch 7/10
584/584 [==============================] - 110s 188ms/step - loss: 1.09